In [58]:
import numpy as np
import random

# Determines the number of times a given number shows up in a given row, column, and calculated square
def numConflicts(sudoku, num, row, col):
  # Count Variable for how many number of conflicts
  count = 0

  # Count how many times it appears in the row
  count += sudoku[row][0:9].count(num)

  # Create a column list
  column = []
  for i in sudoku:
    column.append(i[col])

  # Use the count method to count times appear in the row list
  count += column.count(num)

  # Determine which of the 9 squares we are in by figuring out the starting row and col
  # Use the way that an int is stored to round the number
  # (i.e. row 7 col 4; 4/3 = 1 * 3 = 3 so the square stars at col 3, 7/3=2*3 = 6 so square starts at row 6)
  squareCol = int(col/3) * 3
  squareRow = int(row/3) * 3

  # Using the starting row make a temp object which is just the rows the square has
  temp = sudoku[squareRow:squareRow + 3]

  # Make a list which is the 9 values in the square
  check = []
  for k in range(3):
    for l in range(3):
      check.append(temp[k][squareCol + l])

  # Count number of times the given number appears
  count += check.count(num)

  return count

# Given a list of numbers, determine if any repeat
def repeatNums(nums):
  # Helper variable to see values already checked
  temp = []
  for i in nums:
    # If temp already has an instance of i then return true for a repeat number
    if temp.count(i) > 0 or i ==0:
      return True
    # Else add the number to the list of checked numbers
    else:
      temp.append(i)

  # If gone through the for loop without hitting the first return, then return false
  return False

# Determine if the array is a finished sudoku
def finishCheck(sudoku):
  # check the Rows
  for i in range(len(sudoku)):
    # Use repeat nums and just pass the row in
    if repeatNums(sudoku[i]):
      return False

  # Check columns
  # For loop for going through each column of the sudoku
  for j in range(len(sudoku[0])):

    # Helper variable for making a list of numbers in the column
    temp = []

    # For loop to add all values in a column to the helper variable
    for k in range(len(sudoku)):
      temp.append(sudoku[k][j])

    # Use repeat nums to check current column and if there are repeat nums then return false
    if repeatNums(temp):
      return False

  # Check the Squares
  # Similar to the square in numConflicts
  for i in range(3):
    # i * 3 allows for x to skip from 0 to 3 to 6 which is the starting rows for the squares
    x = i * 3
    temp = sudoku[x:x+3]

    for j in range(3):
      y = j * 3

      # Helper function to keep track of the numbers in the current square
      check = []
      for k in range(3):
        for l in range(3):
          check.append(temp[k][y + l])

      # Use repeatNums to check the helper function with the numbers in the current square
      if repeatNums(check):
        return False

  return True

# Return a list of numbers that are likely in a given space
def likelyNums(sudoku, row, col):
  # Nums is the list of likely numbers to return
  nums = []

  # minColnflicts stores how many conflicts each number (1-9) has in given space
  minConflicts = np.zeros(9)
  for i in range(len(minConflicts)):

    # Repeated call to num conflicts that checks each number (i+1 because i 0s 0-8)
    minConflicts[i] = numConflicts(sudoku, i + 1, row, col)

  # minColflicts contains how many conflicts each number has, use min function to find the minConflicts
  # Then use a for loop to find each instance of the minimum, and add to nums
  minVal = min(minConflicts)
  for i in range(len(minConflicts)):
    if minVal == minConflicts[i]:
      # i + 1 because i is 0-8 but we want 1-9
      nums.append(i + 1)

  return nums

# Return a list of numbers that are likely in a given space, with a list of possible numbers
def likelyNumsInList(sudoku, row, col, given):
  # Nums is the list of likely numbers to return
  nums = []

  # minColnflicts stores how many conflicts each number (1-9) has in given space
  minConflicts = np.zeros(9)
  for i in range(len(minConflicts)):

    # Repeated call to num conflicts that checks each number (i+1 because i 0s 0-8)
    minConflicts[i] = numConflicts(sudoku, i + 1, row, col)

  # minColflicts contains how many conflicts each number has, use min function to find the minConflicts
  # Then use a for loop to find each instance of the minimum, and add to nums
  minVal = min(minConflicts)
  for i in range(len(minConflicts)):
    if minVal == minConflicts[i]:
      # i + 1 because i is 0-8 but we want 1-9
      if i+1 in given:
        nums.append(i + 1)

  return nums

# Check if a given number in a given location appears in multiple rows of the same square
# return true when the number appears in other rows of the same square
def inSquareMultiRow(sudoku, row, col, num):

  # Same logic as above for square
  squareCol = int(col/3) * 3
  squareRow = int(row/3) * 3

  for k in range(3):
    for l in range(3):
      # Temp variables to traverse locations in the square
      tempRow = squareRow + k
      tempCol = squareCol + l

      # If the temporary row is the given row or if the number is already filled out then skip
      if tempRow == row or sudoku[tempRow][tempCol] != 0:
        continue

      # Temp variable to get the potential numbers in the current spot
      temp = likelyNums(sudoku,tempRow,tempCol)

      # if the current spot can have the number then return false
      if temp.count(num) > 0:
        return True

  # If we traverse the for loop without hitting the previous return statement then it is true
  return False

# Given a row, go through and change empty numbers in accordance with sudoku logic
def changeRow(original, sudoku, row, rndFill = False):
  # Dictionary that keeps the spot and potential numbers for that spot
  changes = {}

  # loop to make sure the row is set back to normal
  for i in range(len(original[0])):
    if original[row][i] == 0:
      sudoku[row][i] = 0

  # loop to go through each element in the given row
  for j in range(len(original[0])):

    # If we already have a given value in this spot then we continue
    if original[row][j] != 0:
      continue

    # Get a list of numbers likely for this spot
    nums = likelyNums(sudoku, row, j)

    # If there is one number for this spot then set it
    if len(nums) == 1:
      sudoku[row][j] = nums[0]
    # Else we add to our dictionary the spot and potential numbers
    else:
      changes.update({j:nums})
  #if rndFill:
    #print(changes.items())
    #res = [k for k, v in sorted(changes.items(), key=lambda item: len(item[1]))]

  # List of all possible numbers for this row
  numsLeft = [1,2,3,4,5,6,7,8,9]
  to_remove = []
  # Loop through and remove numbers already taken
  for x in sudoku[row]:
    if numsLeft.count(x) > 0:
      numsLeft.remove(x)

  # Next we find a spot for the remaining numbers
  for x in numsLeft:

    # Potential spots is spots where the current number (x) can go in the row
    potentialSpots = []

    # Use the dictionary to find lists where x can go
    for y in changes:
      if changes.get(y).count(x) > 0:
        potentialSpots.append(y)
    # If x appears in one list it must go there
    # i.e. check for when multiple numbers can go in this spot but x only appears in this spot
    # AKA Obvious Single
    if len(potentialSpots) == 1:
      sudoku[row][potentialSpots[0]] = x
      to_remove.append(x)

  # Remove numbers that have already been found
  for x in to_remove:
    numsLeft.remove(x)

  if rndFill:
    print("USING COMPLEX LOGIC")
    temp_remove = {}
    for x in changes:
      temp = changes.get(x)
      for y in temp:
        if not inSquareMultiRow(sudoku, row, x, y) and temp_remove.get(y) == None:
          squareCol = int(x / 3)
          temp_remove.update({y:squareCol})


    if len(temp_remove) > 0:

      for y in temp_remove:
        for x in changes:
          if int(x / 3) == temp_remove.get(y):
            continue

          if changes.get(x).count(y) > 0:

            changes.get(x).remove(y)


    for x in changes:
      temp = changes.get(x)
      if len(temp) == 1:
        print("Naked Pair")
        sudoku[row][x] = temp[0]

  return sudoku

# Works the same as change row
def changeCol(original, sudoku, col):
  changes = {}
  for i in range(len(original[0])):
    if original[i][col] == 0:
      sudoku[i][col] = 0

  for j in range(len(original[0])):

    if original[j][col] != 0:
      continue
    nums = likelyNums(sudoku, j, col)
    if len(nums) == 1:
      sudoku[j][col] = nums[0]
    else:

      changes.update({j:nums})

  numsLeft = [1,2,3,4,5,6,7,8,9]
  spotsChanged = []
  for x in range(len(original[0])):
    if numsLeft.count(sudoku[x][col]) > 0:
      numsLeft.remove(sudoku[x][col])

  for x in numsLeft:
    potentialSpots = []
    for y in changes:
      if changes.get(y).count(x) > 0:
        potentialSpots.append(y)
    if len(potentialSpots) == 1:
      sudoku[potentialSpots[0]][col] = x

  return sudoku

# Checks all squares for any obvious singles or doubles
# Obvious singles is when a number can only go in one spotin a square
# Obvious Doubles is when two numbers (i.e. 7 and 9) are the only numbers in two spots, which then means can be removed from other spots
def obviousSquare(sudoku, workingSudoku):

  for i in range(3):
    for j in range(3):
      squareRow = i
      squareCol = j
      coords = []
      numsLeft = [1,2,3,4,5,6,7,8,9]
      for k in range(3):
        for l in range(3):
          # Temp variables to traverse locations in the square
          tempRow = (squareRow * 3) + k
          tempCol = (squareCol * 3) + l
          #print(sudoku[tempRow][tempCol])

          # If the number is already filled out then skip
          if sudoku[tempRow][tempCol] != 0:
            numsLeft.remove(sudoku[tempRow][tempCol])
            continue
          else:
            tempList = [tempRow, tempCol]
            coords.append(tempList)
      numsFreq = {}
      for n in numsLeft:
        numsFreq.update({n: 0})
      #print(numsLeft)
      for n in coords:
        likelyNums = likelyNumsInList(sudoku, n[0], n[1], numsLeft)
        #print(n, likelyNums)
        if len(likelyNums) == 1:
          workingSudoku[n[0]][n[1]] = likelyNums[0]

        else:
          for z in likelyNums:
            numsFreq[z] = numsFreq[z] + 1




  return workingSudoku

def xWingRow2(sudoku, row, col1, col2, num):
  for i in range(len(sudoku)):
    if i == row:
      continue

    temp = sudoku[i]
    if temp[col1] != 0 or temp[col2] != 0:
      continue

    col1Check = likelyNums(sudoku, i, col1)
    col2Check = likelyNums(sudoku, i, col2)
    if num in col1Check and num in col2Check and abs(i - row) > 3:
      return i

  return -1

def xWingChange(workingSudoku, row1, row2, col1, col2, num):
  if row1 == row2 or col1 == col2:
    return workingSudoku


  for i in range(len(workingSudoku)):
    tempRow = workingSudoku[i]
    if i == row1 or i == row2:
      for j in range(len(tempRow)):
        if tempRow[j] != 0 or j == col1 or j == col2:
          continue

        nums = likelyNums(workingSudoku, i, j)
        if num in nums and len(nums) > 1:
          nums.remove(num)
          if len(nums) == 1:
            workingSudoku[i][j] = nums[0]
            print("XWing num: ", num, " Corners: ", row1, row2, col1, col2)
            print("Number: ", nums[0], "Coords: ", i, j)
            printSudoku(workingSudoku)
            print()

    else:


      c1Check = True
      c2Check = True
      if tempRow[col1] != 0:
        c1Check = False
      if tempRow[col2] != 0:
        c2Check = False

      c1Nums = likelyNums(workingSudoku, i, col1)
      c2Nums = likelyNums(workingSudoku, i, col2)

      if num in c1Nums and len(c1Nums) > 1 and c1Check:
        c1Nums.remove(num)
        if len(c1Nums) == 1:
            workingSudoku[i][col1] = c1Nums[0]
            print("XWing num: ", num, " Corners: ", row1, row2, col1, col2)
            print("Number: ", c1Nums[0], "Coords: ", i, col1)
            printSudoku(workingSudoku)
            print()

      if num in c2Nums and len(c2Nums) > 1 and c2Check:
        c2Nums.remove(num)
        if len(c2Nums) == 1:
            workingSudoku[i][col2] = c2Nums[0]
            print("XWing num: ", num, " Corners: ", row1, row2, col1, col2)
            print("Number: ", c2Nums[0], "Coords: ", i, col2)
            printSudoku(workingSudoku)
            print()


  return workingSudoku

def advancedStrat(sudoku, workingSudoku):
  for k in range(1,10):
    for i in range(len(workingSudoku)):
      row = sudoku[i]
      c1 = -1
      c2 = -1
      for j in range(len(row)):
        if row[j] != 0:
          continue

        nums = likelyNums(workingSudoku, i, j)
        if k in nums:
          if c1 < 0:
            c1 = j
          elif c2 < 0 and abs(j - c1) > 3:
            c2 = j
            r2 = xWingRow2(sudoku, i, c1, c2, k)
            if r2 != -1:
              sudoku=xWingChange(workingSudoku, i, r2, c1, c2, k)
              for i in range(len(workingSudoku)):
                for j in range(len(workingSudoku[0])):
                  sudoku[i][j] = int(workingSudoku[i][j])

          else:
            continue
  return workingSudoku



def printSudoku(sudoku):
  for i in range(len(sudoku)):
    row = sudoku[i]
    tempSTR = " "
    for j in range(len(row)):
      tempSTR += str(row[j]) + " "
      if j % 3 == 2 and j != 8:
        tempSTR += "| "
    print(tempSTR)
    if i % 3 == 2 and i != 8:
      print("-----------------------")


In [70]:
def solveSudoku(sudoku):
  workingSudoku = [[0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0]]

  for i in range(len(workingSudoku)):
    for j in range(len(workingSudoku[0])):
      workingSudoku[i][j] = int(sudoku[i][j])

  print("Given:")
  printSudoku(sudoku)

  additionalLogic = False
  square = False
  row = True
  count = 0
  while not finishCheck(sudoku):

    if row:
      print("Row")
      for i in range(len(sudoku)):
        workingSudoku = changeRow(sudoku, workingSudoku, i, additionalLogic)
        #print()
        #printSudoku(workingSudoku)
      if workingSudoku == sudoku:
        row = False
    elif square:

      print("Square")
      workingSudoku = obviousSquare(sudoku, workingSudoku)
      if workingSudoku == sudoku:
        count = count+1
        square = False
        row = True
      """
      for i in range(len(workingSudoku)):
        for j in range(len(workingSudoku[0])):
          workingSudoku[i][j] = int(sudoku[i][j])
      workingSudoku = advancedStrat(sudoku, workingSudoku)
      additionalLogic = False
      row = True
      #printSudoku(workingSudoku)
      """
    else:
      print("column")
      for i in range(len(sudoku)):
        workingSudoku = changeCol(sudoku, workingSudoku, i)
      if workingSudoku == sudoku:



        square = True

    for i in range(len(workingSudoku)):
      for j in range(len(workingSudoku[0])):
        sudoku[i][j] = workingSudoku[i][j]

    if count == 50:
      print("DEBUG:")
      printSudoku(workingSudoku)
      break


  print("Finished")
  printSudoku(sudoku)

In [66]:
sudoku =          [[0,2,8,0,0,0,7,0,0],
[0,0,7,0,0,0,0,9,3],
[0,0,0,0,0,0,4,0,0],
[0,0,4,0,0,0,3,0,0],
[0,0,0,0,0,5,0,1,0],
[0,8,0,1,6,9,0,0,0],
[0,0,0,0,1,0,0,0,9],
[5,0,0,0,0,0,0,3,0],
[0,0,9,2,0,4,0,0,0]]
"""
sudoku =     [ [0,0,7,0,3,0,2,8,4],
[0,0,0,0,7,0,1,3,9 ],
[0,0,0,0,0,0,6,7,5 ],
[7,6,2,3,5,9,4,1,8 ],
[1,5,4,8,2,7,3,9,6 ],
[9,0,0,0,0,0,7,5,2],
[8,7,6,0,9,0,5,2,3],
[0,0,0,7,6,3,8,4,1],
[3,4,1,2,8,5,9,6,7]]
"""
workingSudoku = [[0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0]]

for i in range(len(workingSudoku)):
  for j in range(len(workingSudoku[0])):
    workingSudoku[i][j] = int(sudoku[i][j])


solveSudoku(sudoku)
#printSudoku(sudoku)
#sudoku = advancedStrat(sudoku, workingSudoku)
#solveSudoku(sudoku)

#print()
#printSudoku(workingSudoku)

Given:
 0 2 8 | 0 0 0 | 7 0 0 
 0 0 7 | 0 0 0 | 0 9 3 
 0 0 0 | 0 0 0 | 4 0 0 
-----------------------
 0 0 4 | 0 0 0 | 3 0 0 
 0 0 0 | 0 0 5 | 0 1 0 
 0 8 0 | 1 6 9 | 0 0 0 
-----------------------
 0 0 0 | 0 1 0 | 0 0 9 
 5 0 0 | 0 0 0 | 0 3 0 
 0 0 9 | 2 0 4 | 0 0 0 
Row









column
column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
Row









column
Square
DEBUG:
 0 2 8 | 0 0 0 | 7 0 0 
 0 0 7 | 0 0 0 | 0 9 3 
 0 0 0 | 0 0 0 | 4 0 0 
-----------------------
 0 0 4 | 0 0 0 | 3 0 0 
 0 0 0 | 0 0 5 | 9 1 0 
 0 8 0 | 1 6 9 | 0 0 0 
-------

In [71]:
sudoku = []
for i in range(9):
  temp = input("Please enter row " + str(i+1) + ":\n")
  string = []
  for j in temp:
    string.append(int(j))
  sudoku.append(string)

solveSudoku(sudoku)

Please enter row 1:
040000015
Please enter row 2:
800300020
Please enter row 3:
701000600
Please enter row 4:
506000030
Please enter row 5:
007200090
Please enter row 6:
000005700
Please enter row 7:
609043800
Please enter row 8:
000620000
Please enter row 9:
300008206
Given:
 0 4 0 | 0 0 0 | 0 1 5 
 8 0 0 | 3 0 0 | 0 2 0 
 7 0 1 | 0 0 0 | 6 0 0 
-----------------------
 5 0 6 | 0 0 0 | 0 3 0 
 0 0 7 | 2 0 0 | 0 9 0 
 0 0 0 | 0 0 5 | 7 0 0 
-----------------------
 6 0 9 | 0 4 3 | 8 0 0 
 0 0 0 | 6 2 0 | 0 0 0 
 3 0 0 | 0 0 8 | 2 0 6 
Row
Row
Row
Row
column
column
column
column
column
column
column
Square
Row
Row
Row
column
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
Square
Row
column
S